In [1]:
# 빅데이터분석기사 실기 유형3 - 카이제곱검정 문제

"""
문제 1: 적합도 검정 (Goodness of Fit Test)

한 주사위가 공정한지 확인하기 위해 600번 던진 결과가 다음과 같다.
유의수준 0.05에서 이 주사위가 공정한지 검정하시오.

면:    1   2   3   4   5   6
관측:  95  98  102 105 96  104

H0: 주사위가 공정하다 (각 면이 나올 확률이 1/6로 동일)
H1: 주사위가 공정하지 않다
"""

import numpy as np
import pandas as pd
from scipy.stats import chi2, chisquare

# 관측 데이터
observed = [95, 98, 102, 105, 96, 104]
faces = [1, 2, 3, 4, 5, 6]
total_throws = sum(observed)

print("주사위 던지기 결과:")
dice_df = pd.DataFrame({
    '면': faces,
    '관측빈도': observed,
    '관측비율': [obs/total_throws for obs in observed]
})
print(dice_df)

# 기댓값 계산 (공정한 주사위라면 각 면이 나올 확률은 1/6)
expected_prob = 1/6
expected = [total_throws * expected_prob] * 6

print(f"\n총 던진 횟수: {total_throws}")
print(f"각 면의 기댓값: {expected[0]:.1f}")

# 카이제곱 통계량 계산
chi2_stat = sum((obs - exp)**2 / exp for obs, exp in zip(observed, expected))

print(f"\n카이제곱 통계량: {chi2_stat:.4f}")

# scipy를 이용한 검정
chi2_scipy, p_value = chisquare(observed)
print(f"scipy 카이제곱 통계량: {chi2_scipy:.4f}")
print(f"p-value: {p_value:.6f}")

# 자유도와 임계값
df = len(observed) - 1  # 자유도 = 범주 수 - 1
alpha = 0.05
chi2_critical = chi2.ppf(1 - alpha, df)

print(f"자유도: {df}")
print(f"임계값 (α=0.05): {chi2_critical:.4f}")

# 결론
print(f"\n=== 검정 결과 (α = {alpha}) ===")
if p_value < alpha:
    print(f"p-value({p_value:.6f}) < α({alpha}) → 귀무가설 기각")
    print("주사위가 공정하지 않다고 할 수 있다.")
else:
    print(f"p-value({p_value:.6f}) >= α({alpha}) → 귀무가설 채택")
    print("주사위가 공정하다고 할 수 있다.")

if chi2_stat > chi2_critical:
    print(f"χ²({chi2_stat:.4f}) > 임계값({chi2_critical:.4f}) → 귀무가설 기각")
else:
    print(f"χ²({chi2_stat:.4f}) <= 임계값({chi2_critical:.4f}) → 귀무가설 채택")


주사위 던지기 결과:
   면  관측빈도      관측비율
0  1    95  0.158333
1  2    98  0.163333
2  3   102  0.170000
3  4   105  0.175000
4  5    96  0.160000
5  6   104  0.173333

총 던진 횟수: 600
각 면의 기댓값: 100.0

카이제곱 통계량: 0.9000
scipy 카이제곱 통계량: 0.9000
p-value: 0.970222
자유도: 5
임계값 (α=0.05): 11.0705

=== 검정 결과 (α = 0.05) ===
p-value(0.970222) >= α(0.05) → 귀무가설 채택
주사위가 공정하다고 할 수 있다.
χ²(0.9000) <= 임계값(11.0705) → 귀무가설 채택


In [2]:
"""
문제 2: 독립성 검정 (Test of Independence)

한 회사에서 직급과 교육 프로그램 참여 의향 간의 관련성을 조사했다.
유의수준 0.05에서 직급과 교육 프로그램 참여 의향이 독립인지 검정하시오.

분할표:
              참여    불참여   계
사원           45      15     60
대리           35      25     60  
과장           25      35     60
부장           15      45     60
계            120     120    240

H0: 직급과 교육 프로그램 참여 의향은 독립이다
H1: 직급과 교육 프로그램 참여 의향은 독립이 아니다
"""

from scipy.stats import chi2_contingency

# 분할표 데이터
contingency_table = np.array([
    [45, 15],  # 사원
    [35, 25],  # 대리
    [25, 35],  # 과장
    [15, 45]   # 부장
])

# 행과 열 이름
positions = ['사원', '대리', '과장', '부장']
participation = ['참여', '불참여']

print("분할표:")
contingency_df = pd.DataFrame(contingency_table, 
                            index=positions, 
                            columns=participation)
# 행 합계와 열 합계 추가
contingency_df['계'] = contingency_df.sum(axis=1)
contingency_df.loc['계'] = contingency_df.sum()
print(contingency_df)

# 카이제곱 독립성 검정
chi2_stat, p_value, dof, expected = chi2_contingency(contingency_table)

print(f"\n=== 카이제곱 독립성 검정 결과 ===")
print(f"카이제곱 통계량: {chi2_stat:.4f}")
print(f"p-value: {p_value:.6f}")
print(f"자유도: {dof}")

# 기댓값 출력
print("\n기댓값:")
expected_df = pd.DataFrame(expected, 
                         index=positions, 
                         columns=participation)
print(expected_df.round(2))

# 임계값
alpha = 0.05
chi2_critical = chi2.ppf(1 - alpha, dof)
print(f"\n임계값 (α=0.05): {chi2_critical:.4f}")

# 각 셀의 기여도 계산
contributions = (contingency_table - expected)**2 / expected
print("\n각 셀의 카이제곱 기여도:")
contrib_df = pd.DataFrame(contributions, 
                        index=positions, 
                        columns=participation)
print(contrib_df.round(4))

# 결론
print(f"\n=== 검정 결과 (α = {alpha}) ===")
if p_value < alpha:
    print(f"p-value({p_value:.6f}) < α({alpha}) → 귀무가설 기각")
    print("직급과 교육 프로그램 참여 의향은 독립이 아니다.")
    print("즉, 직급에 따라 교육 프로그램 참여 의향이 다르다고 할 수 있다.")
else:
    print(f"p-value({p_value:.6f}) >= α({alpha}) → 귀무가설 채택")
    print("직급과 교육 프로그램 참여 의향은 독립이다.")
    print("즉, 직급과 교육 프로그램 참여 의향 간에 관련성이 없다고 할 수 있다.")


분할표:
     참여  불참여    계
사원   45   15   60
대리   35   25   60
과장   25   35   60
부장   15   45   60
계   120  120  240

=== 카이제곱 독립성 검정 결과 ===
카이제곱 통계량: 33.3333
p-value: 0.000000
자유도: 3

기댓값:
      참여   불참여
사원  30.0  30.0
대리  30.0  30.0
과장  30.0  30.0
부장  30.0  30.0

임계값 (α=0.05): 7.8147

각 셀의 카이제곱 기여도:
        참여     불참여
사원  7.5000  7.5000
대리  0.8333  0.8333
과장  0.8333  0.8333
부장  7.5000  7.5000

=== 검정 결과 (α = 0.05) ===
p-value(0.000000) < α(0.05) → 귀무가설 기각
직급과 교육 프로그램 참여 의향은 독립이 아니다.
즉, 직급에 따라 교육 프로그램 참여 의향이 다르다고 할 수 있다.


In [3]:
"""
문제 3: 동질성 검정 (Test of Homogeneity)

세 개의 서로 다른 지역(A, B, C)에서 특정 제품에 대한 선호도를 조사했다.
유의수준 0.05에서 세 지역의 제품 선호도 분포가 동일한지 검정하시오.

조사 결과:
        매우만족  만족  보통  불만족  계
지역A      20    30   25    15    90
지역B      15    35   30    20   100  
지역C      25    25   20    20    90
계         60    90   75    55   280

H0: 세 지역의 제품 선호도 분포가 동일하다
H1: 세 지역의 제품 선호도 분포가 동일하지 않다
"""

# 동질성 검정 데이터
homogeneity_table = np.array([
    [20, 30, 25, 15],  # 지역A
    [15, 35, 30, 20],  # 지역B
    [25, 25, 20, 20]   # 지역C
])

regions = ['지역A', '지역B', '지역C']
satisfaction = ['매우만족', '만족', '보통', '불만족']

print("지역별 제품 선호도 조사 결과:")
homogeneity_df = pd.DataFrame(homogeneity_table, 
                            index=regions, 
                            columns=satisfaction)
# 행 합계와 열 합계 추가
homogeneity_df['계'] = homogeneity_df.sum(axis=1)
homogeneity_df.loc['계'] = homogeneity_df.sum()
print(homogeneity_df)

# 카이제곱 동질성 검정 (독립성 검정과 동일한 방법 사용)
chi2_stat, p_value, dof, expected = chi2_contingency(homogeneity_table)

print(f"\n=== 카이제곱 동질성 검정 결과 ===")
print(f"카이제곱 통계량: {chi2_stat:.4f}")
print(f"p-value: {p_value:.6f}")
print(f"자유도: {dof}")

# 기댓값 출력
print("\n기댓값:")
expected_df = pd.DataFrame(expected, 
                         index=regions, 
                         columns=satisfaction)
print(expected_df.round(2))

# 각 지역별 비율 계산
print("\n각 지역별 선호도 비율:")
proportions = homogeneity_table / homogeneity_table.sum(axis=1, keepdims=True)
prop_df = pd.DataFrame(proportions, 
                      index=regions, 
                      columns=satisfaction)
print(prop_df.round(3))

# 임계값
alpha = 0.05
chi2_critical = chi2.ppf(1 - alpha, dof)
print(f"\n임계값 (α=0.05): {chi2_critical:.4f}")

# 결론
print(f"\n=== 검정 결과 (α = {alpha}) ===")
if p_value < alpha:
    print(f"p-value({p_value:.6f}) < α({alpha}) → 귀무가설 기각")
    print("세 지역의 제품 선호도 분포가 동일하지 않다.")
    print("즉, 지역에 따라 제품 선호도가 다르다고 할 수 있다.")
else:
    print(f"p-value({p_value:.6f}) >= α({alpha}) → 귀무가설 채택")
    print("세 지역의 제품 선호도 분포가 동일하다.")
    print("즉, 지역에 관계없이 제품 선호도가 비슷하다고 할 수 있다.")

print("\n=== 참고사항 ===")
print("- 적합도 검정: 관측값이 특정 분포를 따르는지 검정")
print("- 독립성 검정: 두 변수가 서로 독립인지 검정") 
print("- 동질성 검정: 여러 집단의 분포가 동일한지 검정")


지역별 제품 선호도 조사 결과:
     매우만족  만족  보통  불만족    계
지역A    20  30  25   15   90
지역B    15  35  30   20  100
지역C    25  25  20   20   90
계      60  90  75   55  280

=== 카이제곱 동질성 검정 결과 ===
카이제곱 통계량: 6.3110
p-value: 0.389268
자유도: 6

기댓값:
      매우만족     만족     보통    불만족
지역A  19.29  28.93  24.11  17.68
지역B  21.43  32.14  26.79  19.64
지역C  19.29  28.93  24.11  17.68

각 지역별 선호도 비율:
      매우만족     만족     보통    불만족
지역A  0.222  0.333  0.278  0.167
지역B  0.150  0.350  0.300  0.200
지역C  0.278  0.278  0.222  0.222

임계값 (α=0.05): 12.5916

=== 검정 결과 (α = 0.05) ===
p-value(0.389268) >= α(0.05) → 귀무가설 채택
세 지역의 제품 선호도 분포가 동일하다.
즉, 지역에 관계없이 제품 선호도가 비슷하다고 할 수 있다.

=== 참고사항 ===
- 적합도 검정: 관측값이 특정 분포를 따르는지 검정
- 독립성 검정: 두 변수가 서로 독립인지 검정
- 동질성 검정: 여러 집단의 분포가 동일한지 검정
